###### **Step 1: Install Required Libraries**

In [1]:
!pip install neo4j openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3



 ###### **Step 2: Set Up Neo4j Connection and OpenAI API Key**

In [2]:
import os
from neo4j import GraphDatabase
import openai

In [3]:
# Set OpenAI API key as an environment variable
os.environ["OPENAI_API_KEY"] = "skproj-0JeTABUHK05VKjbo09iTHhyKQCem6e2eggp8GBP-pdZNGmOM-1BSVRDsItfGVxCcf6IJet3SfTT3BlbkFJqnVduE0c26aXpOM8_jqCF73uZoPFtzmK5R2OEUOqt4MsIsdwKDgm_ymZCpbKknfPUHyZJ-elAA"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Neo4j connection details (use your Neo4j Sandbox URI and credentials)
NEO4J_URI = "bolt://44.203.246.205:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "similarity-paths-interests"

In [5]:
# Initialize the Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

###### **Step 3: Functions to Convert User Query to Cypher, Query Neo4j, and Generate Response**

In [22]:
# Convert the user query to Cypher using GPT-4
def convert_to_cypher(user_query):
    prompt = f"Convert the following user query into a Cypher query to retrieve relevant movie data from Neo4j graph:\n\nUser Query: {user_query}\n\nCypher Query:"

    response = openai.ChatCompletion.create(
        model="gpt-4", # Changed to gpt-4
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.7
    )


    cypher_query = response.choices[0].message['content'].strip()
    # Extract the actual Cypher query by splitting on the first colon and taking the second part
    cypher_query = cypher_query.split(":", 1)[1].strip() # Added this line to extract only the query part

    # Check if the query starts with a valid Cypher clause and add MATCH if needed
    if not cypher_query.startswith(("MATCH", "OPTIONAL MATCH", "MERGE")):
        cypher_query = "MATCH (a:" + cypher_query  # Prepend 'MATCH (a:' to the query

    return cypher_query

In [17]:
# Query Neo4j using the generated Cypher query
def query_with_cypher(cypher_query):
    with driver.session() as session:
        result = session.run(cypher_query)
        context = "\n".join([f"{record}" for record in result])
        return context

In [18]:
def query_with_context_and_generate_response(user_query):
    # Convert user query to Cypher
    cypher_query = convert_to_cypher(user_query)

    # Fetch relevant context using the Cypher query
    context = query_with_cypher(cypher_query)

    # Form the prompt for GPT-4
    prompt = f"Context:\n{context}\n\nUser Query: {user_query}\nAnswer:"

    # Generate response from OpenAI GPT
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message['content']

###### **Step 4: Run the Demo with a Sample Query**

In [27]:
# Example user query
user_query = "Who acted in The Matrix?"

# Get response from GPT-4 with context from Neo4j
response = query_with_context_and_generate_response(user_query)
print("Response:", response)

Response: The main actors in The Matrix are Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, and Hugo Weaving.
